In [3]:
#!/usr/bin/env python3
"""
BrainNet Viewer File Generator for Gold Standard AD Connectivity Connections
Creates node and edge files for brain visualization of each comparison
"""

import pandas as pd
import numpy as np
import os

def create_brain_visualization_files():
    """
    Creates BrainNet Viewer files for each comparison from gold standard connections
    """
    
    print("CREATING BRAINNET VIEWER FILES FOR GOLD STANDARD CONNECTIONS")
    print("==========================================================")
    
    # =============================================================================
    # 1. SETUP AND DATA LOADING
    # =============================================================================
    
    # Load your gold standard connections
    gold_standard_file = 'C:/Users/theya/Downloads/ADNIFinal/FMRIAnalysis/Comprehensive_Publication_Analysis_Corrected/Gold_Standard_Connections_Only_Corrected.csv'
    
    if not os.path.exists(gold_standard_file):
        print(f"❌ Error: {gold_standard_file} not found!")
        print("Please ensure the corrected gold standard CSV is in the current directory.")
        return
    
    print(f"✅ Loading {gold_standard_file}...")
    df = pd.read_csv(gold_standard_file)
    
    print(f"✅ Loaded {len(df)} gold standard connections")
    
    # =============================================================================
    # 2. DEFINE REFERENCE NODE NAMES (Desikan-Killiany 68)
    # =============================================================================
    
    # Reference node names from BrainNet Viewer (must match exactly)
    reference_node_names = [
        # Left hemisphere (indices 0-33)
        'l.bankssts', 'l.caudalanteriorcingulate', 'l.caudalmiddlefrontal', 'l.cuneus',
        'l.entorhinal', 'l.fusiform', 'l.inferiorparietal', 'l.inferiortemporal',
        'l.isthmuscingulate', 'l.lateraloccipital', 'l.lateralorbitofrontal', 'l.lingual',
        'l.medialorbitofrontal', 'l.middletemporal', 'l.parahippocampal', 'l.paracentral',
        'l.parsopercularis', 'l.parsorbitalis', 'l.parstriangularis', 'l.pericalcarine',
        'l.postcentral', 'l.posteriorcingulate', 'l.precentral', 'l.precuneus',
        'l.rostralanteriorcingulate', 'l.rostralmiddlefrontal', 'l.superiorfrontal', 'l.superiorparietal',
        'l.superiortemporal', 'l.supramarginal', 'l.frontalpole', 'l.temporalpole',
        'l.transversetemporal', 'l.insula',
        # Right hemisphere (indices 34-67)
        'r.bankssts', 'r.caudalanteriorcingulate', 'r.caudalmiddlefrontal', 'r.cuneus',
        'r.entorhinal', 'r.fusiform', 'r.inferiorparietal', 'r.inferiortemporal',
        'r.isthmuscingulate', 'r.lateraloccipital', 'r.lateralorbitofrontal', 'r.lingual',
        'r.medialorbitofrontal', 'r.middletemporal', 'r.parahippocampal', 'r.paracentral',
        'r.parsopercularis', 'r.parsorbitalis', 'r.parstriangularis', 'r.pericalcarine',
        'r.postcentral', 'r.posteriorcingulate', 'r.precentral', 'r.precuneus',
        'r.rostralanteriorcingulate', 'r.rostralmiddlefrontal', 'r.superiorfrontal', 'r.superiorparietal',
        'r.superiortemporal', 'r.supramarginal', 'r.frontalpole', 'r.temporalpole',
        'r.transversetemporal', 'r.insula'
    ]
    
    print(f"✅ Defined {len(reference_node_names)} reference node names")
    
    # =============================================================================
    # 3. CREATE MAPPING FUNCTION
    # =============================================================================
    
    def map_region_name_to_reference(region_name):
        """
        Map your region names (e.g., 'rh-precentral') to reference names (e.g., 'r.precentral')
        """
        if pd.isna(region_name) or region_name == '':
            return None
            
        # Convert 'lh-' to 'l.' and 'rh-' to 'r.'
        if region_name.startswith('lh-'):
            ref_name = 'l.' + region_name[3:]  # Remove 'lh-' and add 'l.'
        elif region_name.startswith('rh-'):
            ref_name = 'r.' + region_name[3:]  # Remove 'rh-' and add 'r.'
        else:
            print(f"⚠️ Warning: Unexpected region name format: {region_name}")
            return None
        
        # Check if the mapped name exists in reference
        if ref_name in reference_node_names:
            return ref_name
        else:
            print(f"⚠️ Warning: Mapped name '{ref_name}' not found in reference nodes")
            return None
    
    # =============================================================================
    # 4. PROCESS EACH COMPARISON
    # =============================================================================
    
    # Create output directory
    output_dir = "BrainNet_Visualization_Files"
    os.makedirs(output_dir, exist_ok=True)
    
    comparisons = ['S1_vs_S2', 'S1_vs_S3', 'S3_vs_S2']
    comparison_labels = {
        'S1_vs_S2': 'Early_vs_Advanced',
        'S1_vs_S3': 'Early_vs_Intermediate', 
        'S3_vs_S2': 'Intermediate_vs_Advanced'
    }
    
    for comparison in comparisons:
        print(f"\n📊 Processing {comparison}...")
        
        # Filter data for this comparison
        comp_data = df[df['Comparison'] == comparison].copy()
        print(f"   Found {len(comp_data)} connections for {comparison}")
        
        if len(comp_data) == 0:
            print(f"   ⚠️ No connections found for {comparison}, skipping...")
            continue
        
        # =============================================================================
        # 5. CREATE ADJACENCY MATRIX
        # =============================================================================
        
        # Initialize 68x68 matrix with zeros
        matrix = np.zeros((68, 68), dtype=int)
        
        # Track which nodes are involved
        involved_nodes = set()
        successful_mappings = 0
        failed_mappings = 0
        
        for idx, row in comp_data.iterrows():
            # Map region names to reference format
            region_a_ref = map_region_name_to_reference(row['RegionA_Name'])
            region_b_ref = map_region_name_to_reference(row['RegionB_Name'])
            
            if region_a_ref is None or region_b_ref is None:
                failed_mappings += 1
                continue
            
            # Get matrix indices
            try:
                idx_a = reference_node_names.index(region_a_ref)
                idx_b = reference_node_names.index(region_b_ref)
                
                # Set connection strength based on effect size
                effect_size = abs(row['Effect_Size'])
                
                # Map effect size to connection strength (1-3 scale for visualization)
                if effect_size >= 1.2:
                    strength = 3  # Strong connection
                elif effect_size >= 1.0:
                    strength = 2  # Medium connection
                else:
                    strength = 1  # Weak connection
                
                # Set matrix values (undirected)
                matrix[idx_a, idx_b] = strength
                matrix[idx_b, idx_a] = strength
                
                # Track involved nodes
                involved_nodes.add(idx_a)
                involved_nodes.add(idx_b)
                
                successful_mappings += 1
                
            except ValueError as e:
                print(f"   ⚠️ Error mapping regions: {region_a_ref}, {region_b_ref}")
                failed_mappings += 1
        
        print(f"   ✅ Successfully mapped {successful_mappings} connections")
        print(f"   ❌ Failed to map {failed_mappings} connections")
        print(f"   🧠 {len(involved_nodes)} nodes involved in connections")
        
        # =============================================================================
        # 6. SAVE EDGE FILE
        # =============================================================================
        
        comp_label = comparison_labels[comparison]
        edge_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.edge')
        
        with open(edge_filename, 'w') as f:
            for row in matrix:
                f.write(' '.join(map(str, row)) + '\n')
        
        print(f"   💾 Saved edge file: {edge_filename}")
        
        # =============================================================================
        # 7. CREATE NODE FILE WITH INVOLVEMENT HIGHLIGHTING
        # =============================================================================
        
        # Read the reference node file
        reference_node_file = "Desikan-Killiany68.node"
        
        if os.path.exists(reference_node_file):
            print(f"   📂 Using reference node file: {reference_node_file}")
            
            # Load reference node coordinates
            try:
                # Read node file (assuming format: x y z color size label)
                node_data = []
                with open(reference_node_file, 'r') as f:
                    for line_num, line in enumerate(f):
                        line = line.strip()
                        if line and not line.startswith('#'):  # Skip comments and empty lines
                            parts = line.split()
                            if len(parts) >= 6:  # Ensure we have enough columns
                                x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
                                color = int(parts[3])
                                size = int(parts[4])
                                label = parts[5] if len(parts) > 5 else f"node_{line_num}"
                                
                                # Highlight nodes involved in connections
                                if line_num in involved_nodes:
                                    color = 2  # Red color for involved nodes
                                    size = 6   # Larger size for involved nodes
                                else:
                                    color = 1  # Default color
                                    size = 3   # Default size
                                
                                node_data.append([x, y, z, color, size, label])
                            else:
                                print(f"   ⚠️ Skipping malformed line {line_num}: {line}")
                
                # Save modified node file
                node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
                with open(node_filename, 'w') as f:
                    for node in node_data:
                        f.write(f"{node[0]:.6f} {node[1]:.6f} {node[2]:.6f} {node[3]} {node[4]} {node[5]}\n")
                
                print(f"   💾 Saved node file: {node_filename}")
                
            except Exception as e:
                print(f"   ❌ Error processing reference node file: {e}")
                print(f"   📝 Creating simple node file instead...")
                
                # Create simple node file if reference doesn't work
                node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
                with open(node_filename, 'w') as f:
                    for i, node_name in enumerate(reference_node_names):
                        # Simple coordinates (you may want to use actual brain coordinates)
                        x = (i % 10) * 10  # Simple grid layout
                        y = (i // 10) * 10
                        z = 0
                        
                        # Highlight involved nodes
                        if i in involved_nodes:
                            color = 2  # Red
                            size = 6
                        else:
                            color = 1  # Default
                            size = 3
                        
                        f.write(f"{x} {y} {z} {color} {size} {node_name}\n")
                
                print(f"   💾 Created simple node file: {node_filename}")
        
        else:
            print(f"   ⚠️ Reference node file not found, creating simple coordinates...")
            
            # Create simple node file
            node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
            with open(node_filename, 'w') as f:
                for i, node_name in enumerate(reference_node_names):
                    # Simple coordinates
                    x = (i % 10) * 10
                    y = (i // 10) * 10
                    z = 0
                    
                    # Highlight involved nodes
                    if i in involved_nodes:
                        color = 2  # Red
                        size = 6
                    else:
                        color = 1  # Default
                        size = 3
                    
                    f.write(f"{x} {y} {z} {color} {size} {node_name}\n")
            
            print(f"   💾 Created simple node file: {node_filename}")
        
        # =============================================================================
        # 8. CREATE SUMMARY FILE
        # =============================================================================
        
        summary_filename = os.path.join(output_dir, f'{comp_label}_Summary.txt')
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write(f"GOLD STANDARD CONNECTIONS SUMMARY: {comparison}\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Total connections: {len(comp_data)}\n")
            f.write(f"Successfully mapped: {successful_mappings}\n")
            f.write(f"Failed mappings: {failed_mappings}\n")
            f.write(f"Nodes involved: {len(involved_nodes)}\n\n")
            
            f.write("CONNECTION DETAILS:\n")
            f.write("-" * 20 + "\n")
            for idx, row in comp_data.iterrows():
                f.write(f"{row['RegionA_Name']} <-> {row['RegionB_Name']} ")
                f.write(f"(d={row['Effect_Size']:.3f}, {row['Direction']})\n")
            
            f.write(f"\nFiles created:\n")
            f.write(f"- {comp_label}_GoldStandard.edge\n")
            f.write(f"- {comp_label}_GoldStandard.node\n")
            f.write(f"- {comp_label}_Summary.txt\n")
        
        print(f"   📋 Created summary: {summary_filename}")
    
    # =============================================================================
    # 9. CREATE MASTER SUMMARY
    # =============================================================================
    
    print(f"\n📋 Creating master summary...")
    
    master_summary = os.path.join(output_dir, "Master_Summary.txt")
    with open(master_summary, 'w', encoding='utf-8') as f:
        f.write("BRAINNET VIEWER FILES FOR GOLD STANDARD AD CONNECTIONS\n")
        f.write("=" * 60 + "\n\n")
        
        f.write("FILES CREATED:\n")
        f.write("-" * 15 + "\n")
        for comparison in comparisons:
            comp_label = comparison_labels[comparison]
            comp_data = df[df['Comparison'] == comparison]
            f.write(f"\n{comparison} ({comp_label}):\n")
            f.write(f"  - {comp_label}_GoldStandard.edge ({len(comp_data)} connections)\n")
            f.write(f"  - {comp_label}_GoldStandard.node (68 nodes)\n")
            f.write(f"  - {comp_label}_Summary.txt\n")
        
        f.write(f"\nUSAGE INSTRUCTIONS:\n")
        f.write("-" * 20 + "\n")
        f.write("1. Open BrainNet Viewer\n")
        f.write("2. Load the .node file for brain regions\n")
        f.write("3. Load the corresponding .edge file for connections\n")
        f.write("4. Involved nodes are highlighted in red with larger size\n")
        f.write("5. Connection strength: 1=weak, 2=medium, 3=strong (based on effect size)\n")
        f.write("\nEFFECT SIZE MAPPING:\n")
        f.write("- Effect Size ≥ 1.2: Strong connection (3)\n")
        f.write("- Effect Size ≥ 1.0: Medium connection (2)\n")
        f.write("- Effect Size < 1.0: Weak connection (1)\n")
    
    print(f"✅ BRAINNET VIEWER FILES COMPLETE!")
    print(f"📁 All files saved to: {output_dir}")
    print(f"📋 Master summary: {master_summary}")

if __name__ == "__main__":
    create_brain_visualization_files()

CREATING BRAINNET VIEWER FILES FOR GOLD STANDARD CONNECTIONS
✅ Loading C:/Users/theya/Downloads/ADNIFinal/FMRIAnalysis/Comprehensive_Publication_Analysis_Corrected/Gold_Standard_Connections_Only_Corrected.csv...
✅ Loaded 53 gold standard connections
✅ Defined 68 reference node names

📊 Processing S1_vs_S2...
   Found 20 connections for S1_vs_S2
   ✅ Successfully mapped 20 connections
   ❌ Failed to map 0 connections
   🧠 17 nodes involved in connections
   💾 Saved edge file: BrainNet_Visualization_Files\Early_vs_Advanced_GoldStandard.edge
   📂 Using reference node file: Desikan-Killiany68.node
   ❌ Error processing reference node file: invalid literal for int() with base 10: '1.000000'
   📝 Creating simple node file instead...
   💾 Created simple node file: BrainNet_Visualization_Files\Early_vs_Advanced_GoldStandard.node
   📋 Created summary: BrainNet_Visualization_Files\Early_vs_Advanced_Summary.txt

📊 Processing S1_vs_S3...
   Found 11 connections for S1_vs_S3
   ✅ Successfully mappe

In [5]:
#!/usr/bin/env python3
"""
BrainNet Viewer File Generator for Gold Standard AD Connectivity Connections
Creates node and edge files for brain visualization of each comparison
"""

import pandas as pd
import numpy as np
import os

def create_brain_visualization_files():
    """
    Creates BrainNet Viewer files for each comparison from gold standard connections
    """
    
    print("CREATING BRAINNET VIEWER FILES FOR GOLD STANDARD CONNECTIONS")
    print("==========================================================")
    
    # =============================================================================
    # 1. SETUP AND DATA LOADING
    # =============================================================================
    
    # Load your gold standard connections
    gold_standard_file ='C:/Users/theya/Downloads/ADNIFinal/FMRIAnalysis/Comprehensive_Publication_Analysis_Corrected/Gold_Standard_Connections_Only_Corrected.csv'
    
    if not os.path.exists(gold_standard_file):
        print(f"❌ Error: {gold_standard_file} not found!")
        print("Please ensure the corrected gold standard CSV is in the current directory.")
        return
    
    print(f"✅ Loading {gold_standard_file}...")
    df = pd.read_csv(gold_standard_file)
    
    print(f"✅ Loaded {len(df)} gold standard connections")
    
    # =============================================================================
    # 2. DEFINE REFERENCE NODE NAMES (Desikan-Killiany 68)
    # =============================================================================
    
    # Reference node names from BrainNet Viewer (must match exactly)
    reference_node_names = [
        # Left hemisphere (indices 0-33)
        'l.bankssts', 'l.caudalanteriorcingulate', 'l.caudalmiddlefrontal', 'l.cuneus',
        'l.entorhinal', 'l.fusiform', 'l.inferiorparietal', 'l.inferiortemporal',
        'l.isthmuscingulate', 'l.lateraloccipital', 'l.lateralorbitofrontal', 'l.lingual',
        'l.medialorbitofrontal', 'l.middletemporal', 'l.parahippocampal', 'l.paracentral',
        'l.parsopercularis', 'l.parsorbitalis', 'l.parstriangularis', 'l.pericalcarine',
        'l.postcentral', 'l.posteriorcingulate', 'l.precentral', 'l.precuneus',
        'l.rostralanteriorcingulate', 'l.rostralmiddlefrontal', 'l.superiorfrontal', 'l.superiorparietal',
        'l.superiortemporal', 'l.supramarginal', 'l.frontalpole', 'l.temporalpole',
        'l.transversetemporal', 'l.insula',
        # Right hemisphere (indices 34-67)
        'r.bankssts', 'r.caudalanteriorcingulate', 'r.caudalmiddlefrontal', 'r.cuneus',
        'r.entorhinal', 'r.fusiform', 'r.inferiorparietal', 'r.inferiortemporal',
        'r.isthmuscingulate', 'r.lateraloccipital', 'r.lateralorbitofrontal', 'r.lingual',
        'r.medialorbitofrontal', 'r.middletemporal', 'r.parahippocampal', 'r.paracentral',
        'r.parsopercularis', 'r.parsorbitalis', 'r.parstriangularis', 'r.pericalcarine',
        'r.postcentral', 'r.posteriorcingulate', 'r.precentral', 'r.precuneus',
        'r.rostralanteriorcingulate', 'r.rostralmiddlefrontal', 'r.superiorfrontal', 'r.superiorparietal',
        'r.superiortemporal', 'r.supramarginal', 'r.frontalpole', 'r.temporalpole',
        'r.transversetemporal', 'r.insula'
    ]
    
    print(f"✅ Defined {len(reference_node_names)} reference node names")
    
    # =============================================================================
    # 3. CREATE MAPPING FUNCTION
    # =============================================================================
    
    def map_region_name_to_reference(region_name):
        """
        Map your region names (e.g., 'rh-precentral') to reference names (e.g., 'r.precentral')
        """
        if pd.isna(region_name) or region_name == '':
            return None
            
        # Convert 'lh-' to 'l.' and 'rh-' to 'r.'
        if region_name.startswith('lh-'):
            ref_name = 'l.' + region_name[3:]  # Remove 'lh-' and add 'l.'
        elif region_name.startswith('rh-'):
            ref_name = 'r.' + region_name[3:]  # Remove 'rh-' and add 'r.'
        else:
            print(f"⚠️ Warning: Unexpected region name format: {region_name}")
            return None
        
        # Check if the mapped name exists in reference
        if ref_name in reference_node_names:
            return ref_name
        else:
            print(f"⚠️ Warning: Mapped name '{ref_name}' not found in reference nodes")
            return None
    
    # =============================================================================
    # 4. PROCESS EACH COMPARISON
    # =============================================================================
    
    # Create output directory
    output_dir = "BrainNet_Visualization_Files"
    os.makedirs(output_dir, exist_ok=True)
    
    comparisons = ['S1_vs_S2', 'S1_vs_S3', 'S3_vs_S2']
    comparison_labels = {
        'S1_vs_S2': 'Early_vs_Advanced',
        'S1_vs_S3': 'Early_vs_Intermediate', 
        'S3_vs_S2': 'Intermediate_vs_Advanced'
    }
    
    for comparison in comparisons:
        print(f"\n📊 Processing {comparison}...")
        
        # Filter data for this comparison
        comp_data = df[df['Comparison'] == comparison].copy()
        print(f"   Found {len(comp_data)} connections for {comparison}")
        
        if len(comp_data) == 0:
            print(f"   ⚠️ No connections found for {comparison}, skipping...")
            continue
        
        # =============================================================================
        # 5. CREATE ADJACENCY MATRIX
        # =============================================================================
        
        # Initialize 68x68 matrix with zeros
        matrix = np.zeros((68, 68), dtype=int)
        
        # Track which nodes are involved
        involved_nodes = set()
        successful_mappings = 0
        failed_mappings = 0
        
        for idx, row in comp_data.iterrows():
            # Map region names to reference format
            region_a_ref = map_region_name_to_reference(row['RegionA_Name'])
            region_b_ref = map_region_name_to_reference(row['RegionB_Name'])
            
            if region_a_ref is None or region_b_ref is None:
                failed_mappings += 1
                continue
            
            # Get matrix indices
            try:
                idx_a = reference_node_names.index(region_a_ref)
                idx_b = reference_node_names.index(region_b_ref)
                
                # Set connection strength and color based on effect size and direction
                effect_size = abs(row['Effect_Size'])
                direction = row['Direction']
                
                # Map effect size to connection strength (1-5 scale for better visualization)
                if effect_size >= 1.3:
                    strength = 5  # Very strong connection
                elif effect_size >= 1.1:
                    strength = 4  # Strong connection
                elif effect_size >= 0.9:
                    strength = 3  # Medium connection
                elif effect_size >= 0.8:
                    strength = 2  # Weak connection
                else:
                    strength = 1  # Very weak connection
                
                # Add direction information by using positive/negative values
                if direction == 'Decreased':
                    strength = -strength  # Negative for decreased connectivity
                
                # Create network-specific colors for nodes
                network_a = row.get('RegionA_RSNName', 'Unknown')
                network_b = row.get('RegionB_RSNName', 'Unknown')
                
                # Define network color mapping
                network_colors = {
                    'DMN': 3,   # Red - Default Mode Network
                    'SM': 4,    # Green - Somatomotor  
                    'VA': 5,    # Blue - Ventral Attention
                    'VIS': 6,   # Yellow - Visual
                    'FP': 7,    # Magenta - Frontoparietal
                    'LS': 8,    # Cyan - Limbic System
                    'SUB': 9,   # Orange - Subcortical
                    'DA': 10    # Purple - Dorsal Attention
                }
                
                # Assign colors based on network
                color_a = network_colors.get(network_a, 2)  # Default to red if unknown
                color_b = network_colors.get(network_b, 2)
                
                # Set matrix values (undirected)
                matrix[idx_a, idx_b] = strength
                matrix[idx_b, idx_a] = strength
                
                # Track involved nodes with their networks and colors
                involved_nodes.add((idx_a, color_a, network_a))
                involved_nodes.add((idx_b, color_b, network_b))
                
                successful_mappings += 1
                
            except ValueError as e:
                print(f"   ⚠️ Error mapping regions: {region_a_ref}, {region_b_ref}")
                failed_mappings += 1
        
        print(f"   ✅ Successfully mapped {successful_mappings} connections")
        print(f"   ❌ Failed to map {failed_mappings} connections")
        print(f"   🧠 {len(involved_nodes)} nodes involved in connections")
        
        # =============================================================================
        # 6. SAVE EDGE FILE
        # =============================================================================
        
        comp_label = comparison_labels[comparison]
        edge_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.edge')
        
        with open(edge_filename, 'w') as f:
            for row in matrix:
                f.write(' '.join(map(str, row)) + '\n')
        
        print(f"   💾 Saved edge file: {edge_filename}")
        
        # =============================================================================
        # 7. CREATE NODE FILE WITH INVOLVEMENT HIGHLIGHTING
        # =============================================================================
        
        # Read the reference node file
        reference_node_file = "Desikan-Killiany68.node"
        
        if os.path.exists(reference_node_file):
            print(f"   📂 Using reference node file: {reference_node_file}")
            
            # Load reference node coordinates
            try:
                # Read node file (assuming format: x y z color size label)
                node_data = []
                with open(reference_node_file, 'r') as f:
                    for line_num, line in enumerate(f):
                        line = line.strip()
                        if line and not line.startswith('#'):  # Skip comments and empty lines
                            parts = line.split()
                            if len(parts) >= 6:  # Ensure we have enough columns
                                x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
                                color = int(parts[3])
                                size = int(parts[4])
                                label = parts[5] if len(parts) > 5 else f"node_{line_num}"
                                
                                # Highlight nodes involved in connections
                                if line_num in involved_nodes:
                                    color = 2  # Red color for involved nodes
                                    size = 6   # Larger size for involved nodes
                                else:
                                    color = 1  # Default color
                                    size = 3   # Default size
                                
                                node_data.append([x, y, z, color, size, label])
                            else:
                                print(f"   ⚠️ Skipping malformed line {line_num}: {line}")
                
                # Save modified node file
                node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
                with open(node_filename, 'w') as f:
                    for node in node_data:
                        f.write(f"{node[0]:.6f} {node[1]:.6f} {node[2]:.6f} {node[3]} {node[4]} {node[5]}\n")
                
                print(f"   💾 Saved node file: {node_filename}")
                
            except Exception as e:
                print(f"   ❌ Error processing reference node file: {e}")
                print(f"   📝 Creating simple node file instead...")
                
                # Create simple node file if reference doesn't work
                node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
                with open(node_filename, 'w') as f:
                    for i, node_name in enumerate(reference_node_names):
                        # Simple coordinates (you may want to use actual brain coordinates)
                        x = (i % 10) * 10  # Simple grid layout
                        y = (i // 10) * 10
                        z = 0
                        
                        # Highlight involved nodes
                        if i in involved_nodes:
                            color = 2  # Red
                            size = 6
                        else:
                            color = 1  # Default
                            size = 3
                        
                        f.write(f"{x} {y} {z} {color} {size} {node_name}\n")
                
                print(f"   💾 Created simple node file: {node_filename}")
        
        else:
            print(f"   ⚠️ Reference node file not found, creating simple coordinates...")
            
            # Create simple node file
            node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
            with open(node_filename, 'w') as f:
                for i, node_name in enumerate(reference_node_names):
                    # Simple coordinates
                    x = (i % 10) * 10
                    y = (i // 10) * 10
                    z = 0
                    
                    # Highlight involved nodes
                    if i in involved_nodes:
                        color = 2  # Red
                        size = 6
                    else:
                        color = 1  # Default
                        size = 3
                    
                    f.write(f"{x} {y} {z} {color} {size} {node_name}\n")
            
            print(f"   💾 Created simple node file: {node_filename}")
        
        # =============================================================================
        # 8. CREATE SUMMARY FILE
        # =============================================================================
        
        summary_filename = os.path.join(output_dir, f'{comp_label}_Summary.txt')
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write(f"GOLD STANDARD CONNECTIONS SUMMARY: {comparison}\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Total connections: {len(comp_data)}\n")
            f.write(f"Successfully mapped: {successful_mappings}\n")
            f.write(f"Failed mappings: {failed_mappings}\n")
            f.write(f"Nodes involved: {len(involved_nodes)}\n\n")
            
            f.write("CONNECTION DETAILS:\n")
            f.write("-" * 20 + "\n")
            for idx, row in comp_data.iterrows():
                f.write(f"{row['RegionA_Name']} <-> {row['RegionB_Name']} ")
                f.write(f"(d={row['Effect_Size']:.3f}, {row['Direction']})\n")
            
            f.write(f"\nFiles created:\n")
            f.write(f"- {comp_label}_GoldStandard.edge\n")
            f.write(f"- {comp_label}_GoldStandard.node\n")
            f.write(f"- {comp_label}_Summary.txt\n")
        
        print(f"   📋 Created summary: {summary_filename}")
    
    # =============================================================================
    # 9. CREATE MASTER SUMMARY
    # =============================================================================
    
    print(f"\n📋 Creating master summary...")
    
    master_summary = os.path.join(output_dir, "Master_Summary.txt")
    with open(master_summary, 'w', encoding='utf-8') as f:
        f.write("BRAINNET VIEWER FILES FOR GOLD STANDARD AD CONNECTIONS\n")
        f.write("=" * 60 + "\n\n")
        
        f.write("FILES CREATED:\n")
        f.write("-" * 15 + "\n")
        for comparison in comparisons:
            comp_label = comparison_labels[comparison]
            comp_data = df[df['Comparison'] == comparison]
            f.write(f"\n{comparison} ({comp_label}):\n")
            f.write(f"  - {comp_label}_GoldStandard.edge ({len(comp_data)} connections)\n")
            f.write(f"  - {comp_label}_GoldStandard.node (68 nodes)\n")
            f.write(f"  - {comp_label}_Summary.txt\n")
        
        f.write(f"\nUSAGE INSTRUCTIONS:\n")
        f.write("-" * 20 + "\n")
        f.write("1. Open BrainNet Viewer\n")
        f.write("2. Load the .node file for brain regions\n")
        f.write("3. Load the corresponding .edge file for connections\n")
        f.write("4. Involved nodes are highlighted in red with larger size\n")
        f.write("5. Connection strength: 1=weak, 2=medium, 3=strong (based on effect size)\n")
        f.write("\nEFFECT SIZE MAPPING:\n")
        f.write("- Effect Size ≥ 1.2: Strong connection (3)\n")
        f.write("- Effect Size ≥ 1.0: Medium connection (2)\n")
        f.write("- Effect Size < 1.0: Weak connection (1)\n")
    
    print(f"✅ BRAINNET VIEWER FILES COMPLETE!")
    print(f"📁 All files saved to: {output_dir}")
    print(f"📋 Master summary: {master_summary}")

if __name__ == "__main__":
    create_brain_visualization_files()

CREATING BRAINNET VIEWER FILES FOR GOLD STANDARD CONNECTIONS
✅ Loading C:/Users/theya/Downloads/ADNIFinal/FMRIAnalysis/Comprehensive_Publication_Analysis_Corrected/Gold_Standard_Connections_Only_Corrected.csv...
✅ Loaded 53 gold standard connections
✅ Defined 68 reference node names

📊 Processing S1_vs_S2...
   Found 20 connections for S1_vs_S2
   ✅ Successfully mapped 20 connections
   ❌ Failed to map 0 connections
   🧠 17 nodes involved in connections
   💾 Saved edge file: BrainNet_Visualization_Files\Early_vs_Advanced_GoldStandard.edge
   📂 Using reference node file: Desikan-Killiany68.node
   ❌ Error processing reference node file: invalid literal for int() with base 10: '1.000000'
   📝 Creating simple node file instead...
   💾 Created simple node file: BrainNet_Visualization_Files\Early_vs_Advanced_GoldStandard.node
   📋 Created summary: BrainNet_Visualization_Files\Early_vs_Advanced_Summary.txt

📊 Processing S1_vs_S3...
   Found 11 connections for S1_vs_S3
   ✅ Successfully mappe

In [6]:
#!/usr/bin/env python3
"""
BrainNet Viewer File Generator for Gold Standard AD Connectivity Connections
Creates node and edge files for brain visualization of each comparison
"""

import pandas as pd
import numpy as np
import os

def create_brain_visualization_files():
    """
    Creates BrainNet Viewer files for each comparison from gold standard connections
    """
    
    print("CREATING BRAINNET VIEWER FILES FOR GOLD STANDARD CONNECTIONS")
    print("==========================================================")
    
    # =============================================================================
    # 1. SETUP AND DATA LOADING
    # =============================================================================
    
    # Load your gold standard connections
    gold_standard_file = 'C:/Users/theya/Downloads/ADNIFinal/FMRIAnalysis/Comprehensive_Publication_Analysis_Corrected/Gold_Standard_Connections_Only_Corrected.csv'
    
    if not os.path.exists(gold_standard_file):
        print(f"❌ Error: {gold_standard_file} not found!")
        print("Please ensure the corrected gold standard CSV is in the current directory.")
        return
    
    print(f"✅ Loading {gold_standard_file}...")
    df = pd.read_csv(gold_standard_file)
    
    print(f"✅ Loaded {len(df)} gold standard connections")
    
    # =============================================================================
    # 2. DEFINE REFERENCE NODE NAMES (Desikan-Killiany 68)
    # =============================================================================
    
    # Reference node names from BrainNet Viewer (must match exactly)
    reference_node_names = [
        # Left hemisphere (indices 0-33)
        'l.bankssts', 'l.caudalanteriorcingulate', 'l.caudalmiddlefrontal', 'l.cuneus',
        'l.entorhinal', 'l.fusiform', 'l.inferiorparietal', 'l.inferiortemporal',
        'l.isthmuscingulate', 'l.lateraloccipital', 'l.lateralorbitofrontal', 'l.lingual',
        'l.medialorbitofrontal', 'l.middletemporal', 'l.parahippocampal', 'l.paracentral',
        'l.parsopercularis', 'l.parsorbitalis', 'l.parstriangularis', 'l.pericalcarine',
        'l.postcentral', 'l.posteriorcingulate', 'l.precentral', 'l.precuneus',
        'l.rostralanteriorcingulate', 'l.rostralmiddlefrontal', 'l.superiorfrontal', 'l.superiorparietal',
        'l.superiortemporal', 'l.supramarginal', 'l.frontalpole', 'l.temporalpole',
        'l.transversetemporal', 'l.insula',
        # Right hemisphere (indices 34-67)
        'r.bankssts', 'r.caudalanteriorcingulate', 'r.caudalmiddlefrontal', 'r.cuneus',
        'r.entorhinal', 'r.fusiform', 'r.inferiorparietal', 'r.inferiortemporal',
        'r.isthmuscingulate', 'r.lateraloccipital', 'r.lateralorbitofrontal', 'r.lingual',
        'r.medialorbitofrontal', 'r.middletemporal', 'r.parahippocampal', 'r.paracentral',
        'r.parsopercularis', 'r.parsorbitalis', 'r.parstriangularis', 'r.pericalcarine',
        'r.postcentral', 'r.posteriorcingulate', 'r.precentral', 'r.precuneus',
        'r.rostralanteriorcingulate', 'r.rostralmiddlefrontal', 'r.superiorfrontal', 'r.superiorparietal',
        'r.superiortemporal', 'r.supramarginal', 'r.frontalpole', 'r.temporalpole',
        'r.transversetemporal', 'r.insula'
    ]
    
    print(f"✅ Defined {len(reference_node_names)} reference node names")
    
    # =============================================================================
    # 3. CREATE MAPPING FUNCTION
    # =============================================================================
    
    def map_region_name_to_reference(region_name):
        """
        Map your region names (e.g., 'rh-precentral') to reference names (e.g., 'r.precentral')
        """
        if pd.isna(region_name) or region_name == '':
            return None
            
        # Convert 'lh-' to 'l.' and 'rh-' to 'r.'
        if region_name.startswith('lh-'):
            ref_name = 'l.' + region_name[3:]  # Remove 'lh-' and add 'l.'
        elif region_name.startswith('rh-'):
            ref_name = 'r.' + region_name[3:]  # Remove 'rh-' and add 'r.'
        else:
            print(f"⚠️ Warning: Unexpected region name format: {region_name}")
            return None
        
        # Check if the mapped name exists in reference
        if ref_name in reference_node_names:
            return ref_name
        else:
            print(f"⚠️ Warning: Mapped name '{ref_name}' not found in reference nodes")
            return None
    
    # =============================================================================
    # 4. PROCESS EACH COMPARISON
    # =============================================================================
    
    # Create output directory
    output_dir = "BrainNet_Visualization_Files"
    os.makedirs(output_dir, exist_ok=True)
    
    comparisons = ['S1_vs_S2', 'S1_vs_S3', 'S3_vs_S2']
    comparison_labels = {
        'S1_vs_S2': 'Early_vs_Advanced',
        'S1_vs_S3': 'Early_vs_Intermediate', 
        'S3_vs_S2': 'Intermediate_vs_Advanced'
    }
    
    for comparison in comparisons:
        print(f"\n📊 Processing {comparison}...")
        
        # Filter data for this comparison
        comp_data = df[df['Comparison'] == comparison].copy()
        print(f"   Found {len(comp_data)} connections for {comparison}")
        
        if len(comp_data) == 0:
            print(f"   ⚠️ No connections found for {comparison}, skipping...")
            continue
        
        # =============================================================================
        # 5. CREATE ADJACENCY MATRIX
        # =============================================================================
        
        # Initialize 68x68 matrix with zeros
        matrix = np.zeros((68, 68), dtype=int)
        
        # Track which nodes are involved
        involved_nodes = set()
        successful_mappings = 0
        failed_mappings = 0
        
        for idx, row in comp_data.iterrows():
            # Map region names to reference format
            region_a_ref = map_region_name_to_reference(row['RegionA_Name'])
            region_b_ref = map_region_name_to_reference(row['RegionB_Name'])
            
            if region_a_ref is None or region_b_ref is None:
                failed_mappings += 1
                continue
            
            # Get matrix indices
            try:
                idx_a = reference_node_names.index(region_a_ref)
                idx_b = reference_node_names.index(region_b_ref)
                
                # Set connection strength and color based on effect size and direction
                effect_size = abs(row['Effect_Size'])
                direction = row['Direction']
                
                # Map effect size to connection strength (1-5 scale for better visualization)
                if effect_size >= 1.3:
                    strength = 5  # Very strong connection
                elif effect_size >= 1.1:
                    strength = 4  # Strong connection
                elif effect_size >= 0.9:
                    strength = 3  # Medium connection
                elif effect_size >= 0.8:
                    strength = 2  # Weak connection
                else:
                    strength = 1  # Very weak connection
                
                # Add direction information by using positive/negative values
                if direction == 'Decreased':
                    strength = -strength  # Negative for decreased connectivity
                
                # Create network-specific colors for nodes
                network_a = row.get('RegionA_RSNName', 'Unknown')
                network_b = row.get('RegionB_RSNName', 'Unknown')
                
                # Define network color mapping
                network_colors = {
                    'DMN': 3,   # Red - Default Mode Network
                    'SM': 4,    # Green - Somatomotor  
                    'VA': 5,    # Blue - Ventral Attention
                    'VIS': 6,   # Yellow - Visual
                    'FP': 7,    # Magenta - Frontoparietal
                    'LS': 8,    # Cyan - Limbic System
                    'SUB': 9,   # Orange - Subcortical
                    'DA': 10    # Purple - Dorsal Attention
                }
                
                # Assign colors based on network
                color_a = network_colors.get(network_a, 2)  # Default to red if unknown
                color_b = network_colors.get(network_b, 2)
                
                # Set matrix values (undirected)
                matrix[idx_a, idx_b] = strength
                matrix[idx_b, idx_a] = strength
                
                # Track involved nodes with their networks and colors
                involved_nodes.add((idx_a, color_a, network_a))
                involved_nodes.add((idx_b, color_b, network_b))
                
                successful_mappings += 1
                
            except ValueError as e:
                print(f"   ⚠️ Error mapping regions: {region_a_ref}, {region_b_ref}")
                failed_mappings += 1
        
        print(f"   ✅ Successfully mapped {successful_mappings} connections")
        print(f"   ❌ Failed to map {failed_mappings} connections")
        print(f"   🧠 {len(involved_nodes)} nodes involved in connections")
        
        # Count nodes by network
        network_counts = {}
        for node_idx, color, network in involved_nodes:
            network_counts[network] = network_counts.get(network, 0) + 1
        
        print(f"   🎨 Network distribution: {dict(network_counts)}")
        
        # =============================================================================
        # 6. SAVE EDGE FILE
        # =============================================================================
        
        comp_label = comparison_labels[comparison]
        edge_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.edge')
        
        with open(edge_filename, 'w') as f:
            for row in matrix:
                f.write(' '.join(map(str, row)) + '\n')
        
        print(f"   💾 Saved edge file: {edge_filename}")
        
        # =============================================================================
        # 7. CREATE NODE FILE WITH INVOLVEMENT HIGHLIGHTING
        # =============================================================================
        
        # Read the reference node file
        reference_node_file = "Desikan-Killiany68.node"
        
        if os.path.exists(reference_node_file):
            print(f"   📂 Using reference node file: {reference_node_file}")
            
            # Load reference node coordinates
            try:
                # Read node file (assuming format: x y z color size label)
                node_data = []
                with open(reference_node_file, 'r') as f:
                    for line_num, line in enumerate(f):
                        line = line.strip()
                        if line and not line.startswith('#'):  # Skip comments and empty lines
                            parts = line.split()
                            if len(parts) >= 6:  # Ensure we have enough columns
                                x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
                                color = int(parts[3])
                                size = int(parts[4])
                                label = parts[5] if len(parts) > 5 else f"node_{line_num}"
                                
                                # Create network-based coloring for involved nodes
                                network_colors = {
                                    'DMN': 3, 'SM': 4, 'VA': 5, 'VIS': 6, 
                                    'FP': 7, 'LS': 8, 'SUB': 9, 'DA': 10
                                }
                                
                                # Check if this node is involved in connections
                                node_color = 1   # Default gray
                                node_size = 3    # Default size
                                
                                for node_idx, color, network in involved_nodes:
                                    if node_idx == line_num:
                                        node_color = color
                                        node_size = 8  # Larger for involved nodes
                                        break
                                
                                node_data.append([x, y, z, node_color, node_size, label])
                            else:
                                print(f"   ⚠️ Skipping malformed line {line_num}: {line}")
                
                # Save modified node file
                node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
                with open(node_filename, 'w') as f:
                    for node in node_data:
                        f.write(f"{node[0]:.6f} {node[1]:.6f} {node[2]:.6f} {node[3]} {node[4]} {node[5]}\n")
                
                print(f"   💾 Saved node file: {node_filename}")
                
            except Exception as e:
                print(f"   ❌ Error processing reference node file: {e}")
                print(f"   📝 Creating simple node file instead...")
                
                # Create comprehensive node file with network-based coloring
                node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
                with open(node_filename, 'w', encoding='utf-8') as f:
                    f.write("# Node file for BrainNet Viewer\n")
                    f.write("# Format: X Y Z Color Size Label\n")
                    f.write("# Colors: 1=Gray(default), 3=Red(DMN), 4=Green(SM), 5=Blue(VA), 6=Yellow(VIS), 7=Magenta(FP), 8=Cyan(LS), 9=Orange(SUB), 10=Purple(DA)\n")
                    
                    for i, node_name in enumerate(reference_node_names):
                        # Simple coordinates (you may want to use actual brain coordinates)
                        x = (i % 10) * 10  # Simple grid layout
                        y = (i // 10) * 10
                        z = 0
                        
                        # Find if this node is involved and get its color
                        node_color = 1  # Default gray
                        node_size = 3   # Default size
                        
                        for node_idx, color, network in involved_nodes:
                            if node_idx == i:
                                node_color = color
                                node_size = 8  # Larger for involved nodes
                                break
                        
                        f.write(f"{x} {y} {z} {node_color} {node_size} {node_name}\n")
                
                print(f"   💾 Created network-colored node file: {node_filename}")
        
        else:
            print(f"   ⚠️ Reference node file not found, creating simple coordinates...")
            
            # Create network-colored node file
            node_filename = os.path.join(output_dir, f'{comp_label}_GoldStandard.node')
            with open(node_filename, 'w', encoding='utf-8') as f:
                f.write("# Node file with network-based colors\n")
                f.write("# Colors: 1=Gray(default), 3=Red(DMN), 4=Green(SM), 5=Blue(VA), 6=Yellow(VIS), 7=Magenta(FP), 8=Cyan(LS), 9=Orange(SUB), 10=Purple(DA)\n")
                
                for i, node_name in enumerate(reference_node_names):
                    # Simple coordinates
                    x = (i % 10) * 10
                    y = (i // 10) * 10
                    z = 0
                    
                    # Network-based coloring for involved nodes
                    node_color = 1  # Default gray
                    node_size = 3   # Default size
                    
                    for node_idx, color, network in involved_nodes:
                        if node_idx == i:
                            node_color = color
                            node_size = 8  # Larger for involved nodes
                            break
                    
                    f.write(f"{x} {y} {z} {node_color} {node_size} {node_name}\n")
            
            print(f"   💾 Created network-colored node file: {node_filename}")
        
        # =============================================================================
        # 8. CREATE SUMMARY FILE
        # =============================================================================
        
        summary_filename = os.path.join(output_dir, f'{comp_label}_Summary.txt')
        with open(summary_filename, 'w', encoding='utf-8') as f:
            f.write(f"GOLD STANDARD CONNECTIONS SUMMARY: {comparison}\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Total connections: {len(comp_data)}\n")
            f.write(f"Successfully mapped: {successful_mappings}\n")
            f.write(f"Failed mappings: {failed_mappings}\n")
            f.write(f"Nodes involved: {len(involved_nodes)}\n\n")
            
            f.write("CONNECTION DETAILS:\n")
            f.write("-" * 20 + "\n")
            for idx, row in comp_data.iterrows():
                f.write(f"{row['RegionA_Name']} <-> {row['RegionB_Name']} ")
                f.write(f"(d={row['Effect_Size']:.3f}, {row['Direction']})\n")
            
            f.write(f"\nFiles created:\n")
            f.write(f"- {comp_label}_GoldStandard.edge\n")
            f.write(f"- {comp_label}_GoldStandard.node\n")
            f.write(f"- {comp_label}_Summary.txt\n")
        
        print(f"   📋 Created summary: {summary_filename}")
    
    # =============================================================================
    # 9. CREATE MASTER SUMMARY
    # =============================================================================
    
    print(f"\n📋 Creating master summary...")
    
    master_summary = os.path.join(output_dir, "Master_Summary.txt")
    with open(master_summary, 'w', encoding='utf-8') as f:
        f.write("BRAINNET VIEWER FILES FOR GOLD STANDARD AD CONNECTIONS\n")
        f.write("=" * 60 + "\n\n")
        
        f.write("FILES CREATED:\n")
        f.write("-" * 15 + "\n")
        for comparison in comparisons:
            comp_label = comparison_labels[comparison]
            comp_data = df[df['Comparison'] == comparison]
            f.write(f"\n{comparison} ({comp_label}):\n")
            f.write(f"  - {comp_label}_GoldStandard.edge ({len(comp_data)} connections)\n")
            f.write(f"  - {comp_label}_GoldStandard.node (68 nodes)\n")
            f.write(f"  - {comp_label}_Summary.txt\n")
        
        f.write(f"\nUSAGE INSTRUCTIONS:\n")
        f.write("-" * 20 + "\n")
        f.write("1. Open BrainNet Viewer\n")
        f.write("2. Load the .node file for brain regions\n")
        f.write("3. Load the corresponding .edge file for connections\n")
        f.write("4. Nodes are colored by brain network:\n")
        f.write("   - Red: Default Mode Network (DMN)\n")
        f.write("   - Green: Somatomotor (SM)\n") 
        f.write("   - Blue: Ventral Attention (VA)\n")
        f.write("   - Yellow: Visual (VIS)\n")
        f.write("   - Magenta: Frontoparietal (FP)\n")
        f.write("   - Cyan: Limbic System (LS)\n")
        f.write("   - Orange: Subcortical (SUB)\n")
        f.write("   - Purple: Dorsal Attention (DA)\n")
        f.write("   - Gray: Uninvolved nodes\n")
        f.write("5. Connection strength: Positive=Increased, Negative=Decreased\n")
        f.write("6. Effect size mapping:\n")
        f.write("   - ±5: Very strong (|d| ≥ 1.3)\n")
        f.write("   - ±4: Strong (|d| ≥ 1.1)\n")
        f.write("   - ±3: Medium (|d| ≥ 0.9)\n")
        f.write("   - ±2: Weak (|d| ≥ 0.8)\n")
        f.write("   - ±1: Very weak (|d| < 0.8)\n")
    
    print(f"✅ BRAINNET VIEWER FILES COMPLETE!")
    print(f"📁 All files saved to: {output_dir}")
    print(f"📋 Master summary: {master_summary}")

if __name__ == "__main__":
    create_brain_visualization_files()

CREATING BRAINNET VIEWER FILES FOR GOLD STANDARD CONNECTIONS
✅ Loading C:/Users/theya/Downloads/ADNIFinal/FMRIAnalysis/Comprehensive_Publication_Analysis_Corrected/Gold_Standard_Connections_Only_Corrected.csv...
✅ Loaded 53 gold standard connections
✅ Defined 68 reference node names

📊 Processing S1_vs_S2...
   Found 20 connections for S1_vs_S2
   ✅ Successfully mapped 20 connections
   ❌ Failed to map 0 connections
   🧠 17 nodes involved in connections
   🎨 Network distribution: {'VA': 3, 'DMN': 7, 'LS': 2, 'SM': 2, 'FP': 2, 'SUB': 1}
   💾 Saved edge file: BrainNet_Visualization_Files\Early_vs_Advanced_GoldStandard.edge
   📂 Using reference node file: Desikan-Killiany68.node
   ❌ Error processing reference node file: invalid literal for int() with base 10: '1.000000'
   📝 Creating simple node file instead...
   💾 Created network-colored node file: BrainNet_Visualization_Files\Early_vs_Advanced_GoldStandard.node
   📋 Created summary: BrainNet_Visualization_Files\Early_vs_Advanced_Summa